In [ ]:
MYSQL实现数据分析
1、数据来源https://www.kaggle.com/sdolezel/black-friday
2、SQL无法读取和写入csv的解决办法https://blog.csdn.net/qq_42142315/article/details/84973970

导入数据
use black_friday;
LOAD DATA local INFILE '/Users/jarvis/Downloads/archive/train.csv' 
INTO TABLE original
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"'
LINES TERMINATED BY '\r\n'
IGNORE 1 ROWS;

增加一列id列(alter可以修改字段)
alter table original
add id int primary key auto_increment first;

数据预处理
1、将年龄”55+”->”55”
update original set age = "55"
where age = "55+"

2、检查缺失值
SELECT *
FROM black_friday.original
where User_ID is null or Product_ID is null or Gender is null or Age is null 
or Occupation is null or City_Category is null or Stay_In_Current_City_Years is null or Marital_Status is null or Purchase is null
结论：无

3、查找重复值
SELECT User_ID , Product_ID
FROM black_friday.original 
group by User_ID, Product_ID
having count(*)>1
结论：无

数据分析
1、按照每个客户的客单价降序排列，并将其保存为视图view
drop view if exists Unit_price_per_customer ;
create view Unit_price_per_customer as 
(
	select User_ID,sum(Purchase) as Sum_Purchase
    from black_friday.original
    group by User_ID
    order by Sum_Purchase desc
);

2、获得前20%的客户的销售额占总销售额的比重
总：SELECT sum(Sum_Purchase) FROM black_friday.unit_price_per_customer;
5095812742
前20%：2819645106
占比：55.3%
结论：总体而言，客户的购买力还算是比较均衡的，并没有出现二八分化。

3、用户画像
3.1 男性与女性消费能力比较
SELECT Gender, sum(Purchase), sum(Purchase)/count(*) as purchase_per_gender FROM black_friday.original
group by Gender
结论：男性消费能力要比女性强

3.2 年龄阶段统计
SELECT Age,sum(Purchase), count(*) as number_of_people,
sum(Purchase)/count(*) as purchase_per_gender FROM black_friday.original
group by Age
order by Age

结论：消费主力人群是26-35岁，宽泛点也包括在了18-45岁的群体。性别男性更具有消费优势。

SELECT Age,Gender,sum(Purchase)
FROM black_friday.original
group by Age,Gender
order by Age 

3.3 城市统计
SELECT City_Category,sum(Purchase)
FROM black_friday.original
group by City_Category
order by sum(Purchase)


3.4 职业
SELECT Occupation,sum(Purchase)
FROM black_friday.original
group by Occupation
order by sum(Purchase) desc


3.5 产品分类
SELECT Product_Category_1,Product_Category_2,Product_Category_3,sum(Purchase)
FROM black_friday.original
where Product_Category_1<>0 and Product_Category_2<>0 and Product_Category_3<>0
group by Product_Category_1,Product_Category_2,Product_Category_3
order by sum(Purchase) desc
limit 10
结论：产品分类的组合上，1-2-15的组合卖的最好。


3.6 产品
SELECT Product_ID,sum(Purchase)
FROM black_friday.original
group by Product_ID
order by sum(Purchase) desc
limit 20
结论：产品ID为P00025442的销量最好
